In [1]:
import pandas as pd
from tables import *
import csv
import numpy as np
import pickle
import time
from scipy import sparse
from tqdm import tqdm_notebook as tqdm

In [2]:
path = "/mnt/disks/sdc/data/"


In [3]:
with open(path+'IDs_network.p','rb') as f:
    IDs_network = pickle.load(f)
authors_network = {}
count = 0
with open(path+'author_network.npy','rb') as f:
    while True:
        try:
            authors_network[IDs_network[count]] = np.load(f)
            count += 1
        except ValueError:
            break
del IDs_network


OSError: Failed to interpret file <_io.BufferedReader name='/mnt/disks/sdc/data/author_network.npy'> as a pickle

In [4]:
articles = {}
with open(path+'article.txt','r') as f:
    reader = csv.reader(f,delimiter='\t')
    next(reader)
    for line in reader:
        articles[int(line[0])] = int(line[2])

In [5]:
with open(path+'dict_cite.p','rb') as f:
    dict_cite = pickle.load(f)
with open(path+'dict_citant.p','rb') as f:
    dict_citant = pickle.load(f)

In [6]:
with open(path+'dict_cluster_art.p','rb') as f:
    dict_cluster_art = pickle.load(f)
with open(path+'dict_cluster_ID.p','rb') as f:
    dict_cluster_ID = pickle.load(f)

In [7]:
with open(path+'authors_disc.p','rb') as f:
    authors_disc_idx = pickle.load(f)
list_disciplines = list(authors_disc_idx.keys())
nb_disciplines = len(list_disciplines)

In [8]:
authors_info = {}
with open(path+'authors_info.csv','r') as f:
    reader = csv.reader(f,delimiter='\t')
    next(reader)
    for line in reader:
        authors_info[int(line[0])] = [line[1],int(line[2])]

In [9]:
class SimStats(IsDescription):
    Citant = UInt32Col()
    Cite = UInt32Col()
    Sim = Float16Col()
    TypeCit = UInt8Col()
    AutCitant = UInt32Col()

In [30]:
h5file_stats = open_file(path + 'sim_statistics.h5', mode="w", title="sim_stats")
group = h5file_stats.create_group("/", 'sim')
for j in tqdm(range(len(list_disciplines))):
    disc = list_disciplines[j]
    h5file_stats.create_table(group, disc, SimStats)


/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


A Jupyter Widget

/home/User1/anaconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'Biomedical Research'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/home/User1/anaconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'Clinical Medicine'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/home/User1/anaconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'Earth and Space'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()``

In [33]:
table_stats.flush()
h5file_stats.close()

In [15]:
h5file_sim = open_file(path + 'similarity_refs_only.h5', mode="r", title="similarity")
table_sim = h5file_sim.root.citations.sim
nb_citations = len(table_sim)

In [22]:
import time

In [31]:
citations = range(nb_citations)
nb_citations = len(citations)
idx = np.arange(nb_citations)
np.random.shuffle(idx)
total_iterations = nb_citations

In [32]:
#Type citations
#0:self, 1: network, 2:other
f = open(path+'progress.txt','w')
start_time = time.time()
for i in range(nb_citations):
    id_cit = idx[i]
    line = table_sim[id_cit]
    art_citant = line['Citant']
    art_cite = line['Cite']
    if art_citant in dict_cluster_art and art_cite in dict_cluster_art:
        authors_citant = set(dict_cluster_art[art_citant])
        authors_cite = set(dict_cluster_art[art_cite])
        year_citant = articles[art_citant]
        for aut_citant in authors_citant:
            info = authors_info[aut_citant]
            disc = info[0]
            if aut_citant in authors_cite:
                type_cit = 0
            else:
                if aut_citant in authors_network:
                    network = authors_network[aut_citant]
                    year_network = set(network[0,np.where(network[1,:]<=year_citant)[0]])
                    int_network = year_network.intersection(authors_cite)
                    if len(int_network) > 0:
                        type_cit = 1
                    else:
                        type_cit = 2
                else:
                    type_cit = 2
            table_stats = h5file_stats.get_node('/sim/{}'.format(disc))
            row = table_stats.row
            row['Citant'] = art_citant
            row['Cite'] = art_cite
            row['Sim'] = line['Sim']
            row['TypeCit'] = type_cit
            row['AutCitant'] = aut_citant
            row.append()
            table_stats.flush()
    if i % 20000 == 1 :
        elapsed_time = time.time() - start_time
        elapsed_time_h = np.round(elapsed_time/3600,2)
        time_per_art = elapsed_time/i
        time_left_h = np.round(time_per_art*(total_iterations-i)/3600,2)
        perc = np.round(i/total_iterations*100,3)
        f.write('Progress: {} articles, {}%, Time since start: {}, Time left: {}\n'.format(i,perc,elapsed_time_h,time_left_h))    
        f.flush()

KeyboardInterrupt: 